In [1]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.preprocessing import StandardScaler, QuantileTransformer

In [2]:
# as the code is loaded from a subfolder, we use the following snippet to add meatcube2 to the import path
# for a normal usage with meatcube2 installed, the two following lines are unnecessary
import sys, os
try:
    current_folder = os.path.dirname(__file__) # normal way
except NameError:
    current_folder = globals()['_dh'][0] # jupyter notebook way
sys.path.append(os.path.join(current_folder, '../..'))

# we load meatcube2 
from meatcube2.models.meatcube_torch import MeATCubeCB
from meatcube2.models.ct_coat import CtCoAT
from meatcube2.models.ct_coat_naive import CtCoAT as CtCoATNaive
from meatcube2.models.AbstractEnergyBasedClassifier import ACaseBaseEnergyClassifier

In [3]:
from scipy.linalg import norm
from scipy.spatial.distance import euclidean, cosine, correlation, braycurtis, hamming
def euclidean_sim(x1,x2):
    return np.exp(-euclidean(x1,x2))
def cosine_sim(x1,x2):
    return np.exp(-cosine(x1,x2))
def correlation_sim(x1,x2):
    return np.exp(-correlation(x1,x2))
def hamming_sim(x1,x2):
    return np.exp(-hamming(x1,x2))
def radius_sim(x1,x2):
    return np.exp(-abs(norm(x1)-norm(x2)))
def class_equality_sim(y1,y2):
    return np.equal(y1,y2).astype(float)

In [4]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [5]:
from sklearn.datasets import make_circles, make_classification, make_moons

# some linearly separable data with added noise
X, y = make_classification(
    n_features=2, n_redundant=0, n_informative=2, random_state=1, n_clusters_per_class=1
)
rng = np.random.RandomState(2)
X += 2 * rng.uniform(size=X.shape)
linearly_separable = (X, y)


# our three datasets
dataset_names = [
    "Moons",
    "Circles",
    "Line",
]
datasets = [
    make_moons(noise=0.3, random_state=0),
    make_circles(noise=0.2, factor=0.5, random_state=1),
    linearly_separable,
]

In [6]:
name = "MeATCube ($\sigma_X$: Euclidean, $\sigma_y$: class)"
clf = MeATCubeCB(euclidean_sim, class_equality_sim)

In [7]:
# Split the data
(X, y), dataset_name = datasets[0], dataset_names[0]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=42
)

# Use the train set as the case base
clf.fit(X_train, y_train)

/home/emarquer/anaconda3/envs/meat/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995026/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


MeATCubeCB(sim_X=<function euclidean_sim at 0x7576af32e950>,
           sim_y=<function class_equality_sim at 0x7576af32ecb0>)

In [8]:
from meatcube2.metrics import confidence, loss_functional_values

In [10]:
confidence(clf, X_train, temperature=100)

array([[1.27932929e-04, 1.22599267e-03],
       [1.16318946e-02, 8.77148516e-06],
       [1.34190632e-07, 7.85512171e-02],
       [2.60888009e-05, 8.72625199e-04],
       [5.89492158e-07, 8.27923737e-03],
       [5.34593229e-04, 2.08826996e-04],
       [3.52953370e-02, 1.48303584e-07],
       [5.67650628e-04, 3.55701263e-05],
       [3.31653660e-05, 2.23390429e-03],
       [1.92273924e-03, 1.61433167e-05],
       [1.46777971e-03, 1.89443577e-05],
       [1.65920065e-04, 4.88581423e-04],
       [3.35739631e-02, 1.84320543e-06],
       [1.59002678e-03, 6.04312369e-05],
       [3.10728967e-03, 3.45188702e-05],
       [1.02173906e-06, 3.60083501e-02],
       [8.30066766e-04, 1.11219546e-04],
       [2.19041545e-07, 5.16118274e-02],
       [5.53541071e-02, 5.95416653e-07],
       [1.74878462e-05, 6.38371910e-03],
       [2.27314825e-02, 1.89933939e-06],
       [4.87320026e-03, 7.05750247e-07],
       [9.35897665e-04, 1.10112893e-04],
       [4.58940717e-03, 1.05285565e-06],
       [3.863239